In [2]:
from bs4 import BeautifulSoup
import requests
#from random import random
from time import sleep
#from email.message import EmailMessage
#from collections import namedtuple
#import smtplib
import csv
from datetime import date

In [ ]:
#do the same thing for Data scientist, analyst, engineer, software engineer, 

In [33]:
# variables for all URLs
indeed_url = 'https://www.indeed.com'
job_search_url = 'https://www.indeed.com/jobs?q={}&l={}'
job_search_id = 'Data analyst'

In [5]:
def get_url(title,location):
    url = job_search_url.format(title,location)
    return url

In [34]:
url = get_url(job_search_id,'United States')

# Extract HTML

In [35]:
response = requests.get(url)
response

<Response [200]>

In [36]:
soup = BeautifulSoup(response.text, 'html.parser')
job_cards = soup.find_all('div','jobsearch-SerpJobCard')
len(job_cards)

15

In [9]:
card = job_cards[0]
atag = card.h2.a
title = atag.get('title') #get method is prefered as it handles key not found error

job_desc = indeed_url + atag.get('href')
company_name = card.find('span', 'company').text.strip()


In [10]:
job_desc

'https://www.indeed.com/company/Enlightened/jobs/Data-Analyst-53330da1480565e8?fccid=0d4958dd09ddbb85&vjs=3'

In [11]:
location = card.find('div','recJobLoc').get('data-rc-loc')

In [12]:
sate = location.split(',')[1]
sate

' DC'

In [13]:
posted_date = card.find('span','date').text

In [14]:
today = date.today().strftime('%m/%d/%y')

In [15]:
try:
    salary = card.find('span','salaryText').text.strip()
except AttributeError:
    salary = ''

In [23]:

try:
    job_rating = card.find('span','ratingsContent').text.strip()
except AttributeError:
    job_rating = ''
job_rating

''

# Get job details 

In [ ]:
#List of all the skills

In [37]:
job_details = requests.get(job_desc)
job_details

<Response [200]>

In [ ]:
detail_soup = BeautifulSoup(job_details.text, 'html.parser')

job_desc = detail_soup.find('div','jobsearch-jobDescriptionText').text.strip()

# Get job data

In [45]:
def get_data_from_card(card):
    title = card.h2.a.get('title') #get method is prefered as it handles key not found error
   
   
    company_name = card.find('span', 'company').text.strip()
    location = card.find('div','recJobLoc').get('data-rc-loc')
   
    if(',' in location):
        state = location.split(',')[1].strip()
    else:
        state = location
    
    try:
        salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        salary = ''
    
    
    try:
        job_rating = card.find('span','ratingsContent').text.strip()
    except AttributeError:
        job_rating = ''
        
    posted_date = card.find('span','date').text
    today = date.today().strftime('%m/%d/%y')
    
    job_desc_url = indeed_url + card.h2.a.get('href')
    job_desc = get_job_desc(job_desc_url)
    
    card_data = (job_search_id,title, company_name, location, state, salary, job_rating, posted_date, today,job_desc_url,job_desc)
    return card_data

In [46]:
job_listing_data = []

for card in job_cards:
    job_listing_data.append(get_data_from_card(card))

In [47]:
job_listing_data[5]

('Data analyst',
 'Data Analyst',
 'BlueLabs Analytics',
 'Washington, DC',
 'DC',
 '$67,000 a year',
 '',
 '30+ days ago',
 '03/29/21',
 'https://www.indeed.com/rc/clk?jk=3cb092c49648dfe3&fccid=53b041cb411b8ed4&vjs=3',
 "The Company\nBlueLabs is a leading provider of analytics services and technology for government, business, and campaigns. Founded in 2013 by senior members of the Obama for America re-election campaign team, we help our clients optimize their engagements with individual customers, supporters and stakeholders to achieve their goals.\nToday, our team counts over 40 data scientists, engineers, and strategists from diverse backgrounds who share a passion for using data to solve the world’s greatest social and analytical challenges. We’ve served more than 400 organizations ranging from political campaigns to advocacy groups, unions, government agencies, and international groups, as well as companies in the automotive, travel, consumer packaged goods, entertainment, healthc

# Get data from next page

In [ ]:
while True:
    try:
        next_page_url = indeed_url + soup.find('a', {'aria-label':'Next'}).get('href')
    except AttributeError:
        break
        
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    job_cards = soup.find_all('div','jobsearch-SerpJobCard')
    
    for card in job_cards:
        job_listing_data.append(get_data_from_card(card))

In [48]:
len(job_listing_data)
job_listing_data[10]

('Data analyst',
 'Data Analyst',
 'Junction Point',
 'Remote',
 'Remote',
 '$47,835 - $113,481 a year',
 '',
 'Active 2 days ago',
 '03/29/21',
 'https://www.indeed.com/company/Junction-Point/jobs/Data-Analyst-41717d8e908c8fad?fccid=c722a76f876ecd58&vjs=3',
 "I am looking for experienced data analyst to manage my newly established company in Chicago. I need someone with data management skill as my company will be about managing the taxation file of every residence on the Illinois area. Its a work from home job. I will be working closely with you and also will be providing the training. For skill requirement I am looking for 2 years in Excel and SQL as it's needed for my work.Job Types: Full-time, Part-time, ContractPay: $47,835.00 - $113,481.00 per yearSchedule:8 hour shiftEducation:Bachelor's (Preferred)Experience:SQL: 1 year (Preferred)Microsoft Excel: 1 year (Preferred)Contract Length:1 yearContract Renewal:LikelyFull Time Opportunity:NoWork Location:Fully RemoteCOVID-19 Precaution

## Putting together

In [1]:
from bs4 import BeautifulSoup
import requests
from time import sleep
from datetime import date
import pickle
import time
import sys

In [2]:
#constants

indeed_url = 'https://www.indeed.com'
job_search_url = 'https://www.indeed.com/jobs?q={}&l={}'



In [3]:
def get_url(title,location):
    title = title.replace(' ', '+')
    location = location.replace(' ', '+')
    url = job_search_url.format(title,location)
    return url

def get_job_desc(job_url):
    time.sleep(20)
    job_details = requests.get(job_url)
    detail_soup = BeautifulSoup(job_details.text, 'html.parser')

    job_desc = detail_soup.find('div','jobsearch-jobDescriptionText').text.strip()
    return job_desc


def get_data_from_card(card,job_search_id):
    title = card.h2.a.get('title') #get method is prefered as it handles key not found error
    try:
        company_name = card.find('span', 'company').text.strip()
    except AttributeError:
        company_name=''
    location = card.find('div','recJobLoc').get('data-rc-loc')
   
    if(',' in location):
        state = location.split(',')[1].strip()
    else:
        state = location
    
    try:
        salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        salary = ''
    
    
    try:
        job_rating = card.find('span','ratingsContent').text.strip()
    except AttributeError:
        job_rating = ''
        
    try:
        posted_date = card.find('span','date').text
    except AttributeError:
        posted_date = ''
        
    today = date.today().strftime('%m/%d/%y')
    
    job_desc_url = indeed_url + card.h2.a.get('href')
    job_desc = get_job_desc(job_desc_url)
    
    card_data = (job_search_id,title, company_name, location, state, salary, job_rating, posted_date, today,job_desc_url,job_desc)
    return card_data

In [4]:
def main(title,location):
    
    
    url = get_url(title,location)
    
    while True:
        time.sleep(100)
        response = requests.get(url)
        print(response.status_code)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            job_cards = soup.find_all('div', 'jobsearch-SerpJobCard')
            for card in job_cards:
                job_listing = get_data_from_card(card,title)
                job_listing_data.append(job_listing)
        else:
            break
            
        try:
            url = indeed_url + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
   

In [9]:
#calling the main method for all the job title and all the locations in a list
# save the data in a pickle file

titles = ['Data Analyst','Data Scientist', 'Data Engineer','Business Analyst','Product Analyst', 'Business Intelligence Engineer','Software Engineer', 'Devops Engineer','Project Manager','Product Manager']
US_states = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington",'Washington DC' "Wisconsin", "West Virginia", "Wyoming"]


In [ ]:

title = "Data Analyst"
location = "United States"

job_listing_data = []
try:
    main(title,location)
except Exception as e: 
        print(title,'United States')
        print("Oops!", sys.exc_info()[0], "occurred.")
        print(e) 
        print("***********************************************")
print(title, len(job_listing_data))
createPickleFile(title)

                

200
200
200
200
200
200
200


# Store the list in a pickle file

In [6]:
def createPickleFile(filename):
    
    file_name = filename + 'Indeed_jobs_listing.pkl'
    with open(file_name, 'wb') as f:
        pickle.dump(job_listing_data, f)
    print("pickle created")

In [76]:
with open(file_name, 'rb') as f:
    data = pickle.load(f)
